# ASE-ABACUS | 第一章：使用方法简介


<div style="color:black; background-color:#FFF3E9; border: 1px solid #FFE0C3; border-radius: 10px; margin-bottom:0rem">
    <p style="margin:1rem; padding-left: 1rem; line-height: 2.5;">
        ©️ <b><i>Copyright 2023 @ Authors</i></b><br/>
        <i>作者：量子御坂
            <b>
            <a href="mailto:quantummisaka@mail.pku.edu.cn">*** 📨 </a>
            </b>
        </i>
        <br/>
        <i>日期：2023-09-19</i><br/>
        <i>最近更新：2024-09-20</i><br/>
        <i>共享协议：</a>本作品采用<a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/">知识共享署名-非商业性使用-相同方式共享 4.0 国际许可协议</a>进行许可。</i><br/>
        <i>快速开始：点击上方的</i> <span style="background-color:rgb(85, 91, 228); color:white; padding: 3px; border-radius: 5px;box-shadow: 2px 2px 3px rgba(0, 0, 0, 0.3); font-size:0.75rem;">开始连接</span> <i>按钮，选择 <b><u>bohrium-notebook:2023-05-31 </u>镜像</b> 和任意配置机型即可开始。</i><br/>
    </p>
</div>

通过ASE-ABACUS接口，你可以做到
1. 通过ASE可视化STRU文件
2. 通过ASE生成与转换STRU文件，进行前处理
3. 通过ASE调用ABACUS进行计算，包括SCF，结构优化，过渡态计算，电子结构分析等
4. 通过ASE读取ABACUS计算结果，进行后处理

让我们开始吧！

哦对了，开始之前你可能需要自己准备一个STRU文件，或者，也可以让ASE帮你生成哦

## 安装ASE-ABACUS接口

如下操作将在目前的python环境中安装ASE和ASE-ABACUS接口。

In [ ]:
# running without ! in shell environments
! git clone https://gitlab.com/1041176461/ase-abacus.git
! cd ase-abacus
! pip install .

## 通过 ASE 可视化 STRU 文件

ASE-ABACUS接口的第一个用处即为用于ABACUS计算的STRU文件的快速可视化。这一点可以通过ASE的visualier模块来快速实现。以下模块实现了一个简单的可视化函数，用于可视化STRU文件。


In [1]:
from ase.io import read
from ase.visualize import view
from pathlib import Path
import sys

def get_stru(inputfile=""):
    cs_atoms = None
    if inputfile:
        cs_stru = Path(inputfile)
        cs_atoms = read(cs_stru, format="abacus")
    else:
        cs_atoms = read("STRU", format="abacus")
    return cs_atoms

cs_atoms = get_stru()
view(cs_atoms)

<Popen: returncode: None args: ['/home/james/apps/anaconda3/envs/ase/bin/pyt...>

上述可视化方法采用ASE默认的ASE-GUI进行可视化，但这个组件在Bohrium Notebook上是不可用的。

我们采用ngl作为可视化器。在使用ngl之前需要先pip安装

In [ ]:
%pip install nglview

以下是一个可视化函数的案例

In [ ]:
from ase.io import read
from ase.visualize import view
from pathlib import Path
import sys

def get_stru(inputfile=""):
    cs_atoms = None
    if inputfile:
        cs_stru = Path(inputfile)
        cs_atoms = read(cs_stru, format="abacus")
    else:
        cs_atoms = read("STRU", format="abacus")
    return cs_atoms

cs_atoms = get_stru()
view(cs_atoms， viewer='ngl')

除了脚本之外，基于命令行有一个更简单的办法来可视化STRU文件。即通过ASE的GUI模块来实现。

In [ ]:
! ase -T gui STRU

## 通过 ASE 生成与转换 STRU 文件

ASE-ABACUS接口的一个关键用处就是从转cif，POSCAR，xyz等格式的结构文件转化为ABACUS的STRU格式文件，或者是反向的转换。当然这个过程也可以通过vaspkit开发组目前正在开发的[ATOMKIT](http://vaspkit.cn/index.php/306.html)软件来实现，这是一个正在快速迭代中的跨平台建模与结构转换脚本，截止目前其最新版本为0.6.1。

ABACUS开发组更推荐采用ASE-ABACUS接口来进行操作

### 1 CIF 转 STRU
从CIF文件中读取结构信息并生成用于ABACUS计算的输入文件往往是计算开展的第一步，操作方法如下：

注意：这里的`pp`和`basis`变量只是将赝势和轨道基组文件指定到STRU文件内，并没有真正生成所需的赝势和轨道基组文件，ABACUS实际运行时会到INPUT里面以及环境变量里面去找对应的赝势和轨道基组文件。

注意：赝势和轨道基组文件是从已经设置好的环境变量对应的位置复制过来的（简单想想就知道也不可能是ASE-ABACUS接口自己生成的），所以在生成赝势和轨道基组文件的时候需要指定之前所述的环境变量

In [3]:
from ase.io import read, write
from pathlib import Path

cs_dir = './'
cs_vasp = Path(cs_dir, 'AMS_DATA-92.cif')
cs_atoms = read(cs_vasp, format='cif')
cs_stru = Path(cs_dir, 'STRU')
pp = {'Si':'Si_ONCV_PBE-1.0.upf','O':'O_ONCV_PBE-1.0.upf'}
basis = {'Si':'Si_gga_8au_100Ry_2s2p1d.orb','O':'O_gga_7au_100Ry_2s2p1d.orb'}
write(cs_stru, cs_atoms, format='abacus', pp=pp, basis=basis)

### 2 STRU 转 CIF
计算完毕的STRU文件需要转换成CIF文件，然后才能被VESTA等建模软件读取。转换方法如下：

In [4]:
from ase.io import read, write
from pathlib import Path

cs_dir = './'
cs_stru = Path(cs_dir, 'STRU')
cs_atoms= read( cs_stru, format='abacus')
cs_vasp = Path(cs_dir, 'STRU.cif')
write(cs_vasp, cs_atoms, format='cif')

### 3 POSCAR 转 STRU
VASP计算所得的POSCAR可以通过ASE-ABACUS接口快速地转换成ABACUS计算所需的STRU文件

In [ ]:
from ase.io import read, write
from pathlib import Path

cs_dir = './'
cs_vasp = Path(cs_dir, 'POSCAR')
cs_atoms = read(cs_vasp, format='vasp')
cs_stru = Path(cs_dir, 'STRU')
pp = {'Al':'Al_ONCV_PBE-1.0.upf'}
basis = {'Al':'Al_gga_8au_100Ry_2s2p1d.orb'}
# 如果要设置磁矩，有两个办法
# 1. 直接添加代码，对各原子设置磁矩
# cs_atoms.set_initial_magnetic_moments([1.0,1.0,1.0,1.0])
# 2. 设置每个原子每个方向的磁矩，此处需要使用二维数组、
# cs_atoms.set_initial_magnetic_moments([[1.0,1.0,1.0],[1.0,1.0,2.0],[1.0,1.0,3.0],[1.0,1.0,4.0]])
write(cs_stru, cs_atoms, format='abacus', pp=pp, basis=basis)

### 4 STRU 转 POSCAR
同样，逆过程也是可以进行的

In [ ]:
from ase.io import read, write
from pathlib import Path
from ase.calculators.abacus import Abacus

cs_dir = './'
cs_stru = Path(cs_dir, 'STRU')
cs_atoms= read(cs_stru, format='abacus')
cs_vasp = Path(cs_dir, 'POSCAR')
write(cs_vasp, cs_atoms, format='vasp')

容易发现，CIF/POSCAR 与 STRU 格式的互换是需要在脚本里面指定很多细节变量的，包括结构类型，STRU文件的赝势、基组，以及设置磁矩等。如果是日常的格式转换的话，可以通过将这些变量全部抽离前置，或者是用一个workflow组织起各个小脚本来解决。

实际上，这种重复造轮子的任务是没有太大必要的，[ATOMKIT](http://vaspkit.cn/index.php/306.html)软件可以更好地进行结构转换。

但ASE有它独有的优势，比如我们可以通过读入POSCAR之后调用ABACUS进行计算，或者是，我们可以直接读入ABACUS的输出文件，来读取ABACUS计算的结果。

### 5 STRU到其他结构格式的转换

ASE支持的结构格式多种多样，具体可参照[ase.io-module](https://wiki.fysik.dtu.dk/ase/ase/io/io.html#module-ase.io)。工作流程与上面的例子类似，都是先用`ase.io.read`读入结构（或者多帧结构），再用`ase.io.write`输出目标结构即可。

这些格式转换也大多可以通过[ATOMKIT](http://vaspkit.cn/index.php/306.html)完成

## 通过ASE调用ABACUS进行计算

生成了STRU文件之后，只需要KPT和INPUT文件，就可以应用ABACUS进行计算了，
可以通过ASE-ABACUS接口，即`ase.calculators.abacus`模块来调用ABACUS进行计算，
具体操作如下：

### 1. 设置环境变量

ABACUS支持模守恒赝势，并支持两种基组：PW（平面波基组）和LCAO（数值原子轨道基组），赝势和轨道基组文件的存放方式可以通过环境变量设置，分别为`ABACUS_PP_PATH`和`ABACUS_ORBITAL_PATH`，设置方法如下：

如果只做PW计算，可以不用设置`ABACUS_ORBITAL_PATH`

In [ ]:
#PP=${HOME}/pseudopotentials
#ORB=${HOME}/orbitals
# in shell
# PP=/home/james/example/PP
# ORB=/home/james/example/ORB
# export ABACUS_PP_PATH=${PP}
# export ABACUS_ORBITAL_PATH=${ORB}
# in python
import os
os.environ['ABACUS_PP_PATH'] = '/home/james/example/PP'
os.environ['ABACUS_ORBITAL_PATH'] = '/home/james/example/ORB'
print(os.environ.get('ABACUS_PP_PATH'))
print(os.environ.get('ABACUS_ORBITAL_PATH'))

/home/james/example/PP
/home/james/example/ORB


注：个人不建议使用环境变量设置方法（容易找不到，并且容易忘），建议像下面所示的脚本一样手动指定赝势和轨道存储位置。

### 2. 加入ABACUS计算器

ASE通过`ase.calculators`调用外部软件作为计算器进行计算，在python中引入ABACUS接口的方式为：

In [ ]:
from ase.calculators.abacus import Abacus

### 3. 基于已有STRU文件开始计算

以下是一个读取已有STRU文件，通过ABACUS进行SCF计算，
并基于ASE的优化器进行结构优化计算的示例脚本

In [ ]:
# read STRU file, generate INCAR and KPT file 
# run optimization jobs
import os
from ase.optimize import QuasiNewton, BFGS, LBFGS, FIRE, GPMin
from ase.io import read
from ase.calculators.abacus import Abacus, AbacusProfile

# setting
stru = read('STRU', format='abacus')
optimizer = QuasiNewton
mpi = 16
omp = 1
abacus = 'abacus'
pseudo_dir = "/home/james/example/PP"
basis_dir = "/home/james/example/ORB"
pp = {"Pt": "Pt_ONCV_PBE-1.0.upf"}
basis = {"Pt": "Pt_gga_7au_100Ry_4s2p2d1f.orb"}
kpts = [3, 3, 1] # KPT setting (will be generate next)
# INPUT setting
parameters = {
    'calculation': 'scf',
    'basis_type': 'lcao',
    'ks_solver': 'genelpa',
    'vdw_method': 'd3_bj',
    'nspin' : 2,
    'xc': 'pbe',
    'ecutwfc': 100,
    'kpts': kpts,
    'pp': pp,
    'basis': basis,
    'pseudo_dir': pseudo_dir,
    'basis_dir': basis_dir,
    'smearing_method': 'gau',
    'smearing_sigma': 0.002,
    'mixing_type': 'broyden',
    'scf_thr': 1e-6,
    'cal_force': 1,
    'cal_stress': 1,
    'out_stru': 1,
    'out_chg': 0,
}

# running
os.environ['OMP_NUM_THREADS'] = f'{omp}'
profile = AbacusProfile(f'mpirun -np {mpi} {abacus}')
stru.calc = Abacus(profile=profile, directory="OUT",
                    **parameters)

qn = optimizer(stru, trajectory='opt.traj')
qn.run(fmax=0.05)

## 通过ASE读取ABACUS计算轨迹

ASE也可以读取ABACUS的计算结果，即计算所得的running*.log文件，并将其转化为含有结构，能量，力和维里的文件格式，便于可视化与用于机器学习势函数训练。

以下是一个读取ABACUS计算结果并将其转换为extxyz格式的例子。extxyz即extended xyz格式文件，是ASE最推荐的文件格式，它可以在存储结构的同时存储其能量、力和维里的计算结果，同时该格式文件和轨迹支持所有能可视化xyz格式的软件进行可视化操作，比如VMD和Ovito。

In [ ]:
from ase.io import read, write
from pathlib import Path
from ase.calculators.abacus import Abacus

cs_dir = './OUT.ABACUS/'
cs_stru = Path(cs_dir, 'running_relax.log')
cs_atoms= read(cs_stru, format='abacus-out')
cs_vasp = Path(cs_dir, 'relax_traj.extxyz')
write(cs_vasp, cs_atoms, format='extxyz')

## ASE-ABACUS 做 NEB 计算

ASE-ABACUS联动不仅可以完成计算过程的前处理、后处理和结构优化计算，还可以调用ASE里面丰富的各种结构演化模块进行其他任务的计算，比如分子动力学（MD），遗传算法（GA），过渡态（TS）搜索等。

过渡态搜索中，最常用的一类方法是NEB方法，进行NEB计算的基本思路如下：
1. 初始化NEB链的初末态，并优化
2. 给出NEB链的中间态插值初猜
3. 进行迭代的NEB链优化计算

ASE-ABACUS进行NEB计算的思路也是类似的。

具体内容参见下期notebook：https://bohrium.dp.tech/notebooks/39369325971
